# SI 618 WN Project Part I

Version 2025.02.06.2.CT

The instructions for this component of the project are available as a 
[Google Doc](https://docs.google.com/document/d/1N2TlYnQuLO3ihIMcr9yKreiSVmqlrOZh7sq6zVcqEnk/edit?tab=t.0#heading=h.hi1g9fls4kkt).
You will need to add multiple Code and Markdown cells below.  I have replicated the
section headings as markdown blocks below.  You will need to add more code and 
markdown blocks below.

Remember to save your work often!


## Project Title
> Provide a descriptive working title for your project.


## Team Members
> List each team member and include their uniqname
> Xiao Dong dongxiao

## Overview
> Give a high level description of your project


## Motivation 
> Explain why you chose this particular topic for your project.	Include the three "real-world" questions that you generated about the data, and be sure to explain what you hope to learn by answering them.


## Data Sources
> List the two (or more) sources of data that you'll be using.  Provide URLs where appropriate.	Explain how the two (or more) datasets complement each other
	


## Data Description
> List the variables of interest, the size of the data sets, missing values, etc.
	


## Data Manipulation
> Mostly code in this section.  This is where you merge your data sets, as well as create new columns (if appropriate)
	


## Data Visualization
> Be sure to include interpretations of your visualizations -- what patterns or anomalies do you see?
